# Uniform Manifold Approximation and Projection (UMAP)
UMAP is a dimensionality reduction algorithm which performs non-linear dimension reduction. 
- It can also be used for visualization of the dataset. 

The UMAP model implemented in cuml allows the user to set the following parameter values:
1.	`n_neighbors`: number of neighboring samples used for manifold approximation. Larger values result in more global views of the manifold, while smaller values result in more local data being preserved (default = 15)
2.	`n_components`: the dimension of the space to embed into (default = 2)
3.	`n_epochs`: number of training epochs to be used in optimizing the low dimensional embedding (default = None)
4.	`learning_rate`: initial learning rate for the embedding optimization (default = 1.0)
5.	`init`: the low dimensional embedding to use: a. 'spectral': use a spectral embedding of the fuzzy 1-skeleton b. 'random': assign initial embedding positions at random (default = 'spectral')
6.	`min_dist`: the minimum distance that should be present between embedded points (default = 0.1)
7.	`spread`: determines how clustered the embedded points will be (default = 1.0)
8.	`set_op_mix_ratio`: is the ratio of pure fuzzy union to intersection. If the value is 1.0 then it will be a pure fuzzy union and for the value of 0.0 it will be a pure fuzzy interpolation (default = 1.0)
9.	`local_connectivity`: number of nearest neighbors that should be assumed to be connected at a local level. It should be not more than the local intrinsic dimension of the manifold (default = 1)
10.	`repulsion_strength`: weighting applied to negative samples in low dimensional embedding optimization. Values > 1 implements a higher negative value to the samples (default = 1.0)
11.	`negative_sample_rate`: the rate at which the negative samples should be selected per positive sample during the optimization process (default = 5)
12.	`transform_queue_size`: embedding new points using a trained model_ will control how aggressively to search for nearest neighbors (default = 4.0)
13.	`verbose`: bool (default False)

The cuml implemetation of the UMAP model has the following functions that one can run:
1.	`fit`: it fits the dataset into an embedded space
2.	`fit_transform`: it fits the dataset into an embedded space and returns the transformed output
3.	transfo`rm: it transforms the dataset into an existing embedded space and returns the low dimensional output

The model accepts only numpy arrays or cudf dataframes as the input. 
- In order to convert your dataset to cudf format please read the cudf [documentation](https://rapidsai.github.io/projects/cudf/en/latest/) 
- For additional information on the UMAP model please refer to the cuml [UMAP documentation](https://rapidsai.github.io/projects/cuml/en/0.6.0/api.html#cuml.UMAP) 

In [1]:
!wget -nc https://github.com/rapidsai/notebooks-extended/raw/master/utils/rapids-colab.sh
!bash rapids-colab.sh

import sys, os

sys.path.append('/usr/local/lib/python3.6/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

--2019-07-27 10:39:57--  https://github.com/rapidsai/notebooks-extended/raw/master/utils/rapids-colab.sh
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rapidsai/notebooks-extended/master/utils/rapids-colab.sh [following]
--2019-07-27 10:39:57--  https://raw.githubusercontent.com/rapidsai/notebooks-extended/master/utils/rapids-colab.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1548 (1.5K) [text/plain]
Saving to: ‘rapids-colab.sh’

rapids-colab.sh     100%[===================>]   1.51K  --.-KB/s    in 0s      

2019-07-27 10:39:58 (229 MB/s) - ‘rapids-colab.sh’ saved [1548/1548]

--2019-07-27 10

In [0]:
import numpy as np
import pandas as pd

import cudf
import os

from sklearn import datasets
from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import KMeans
from sklearn.manifold.t_sne import trustworthiness

from cuml.manifold.umap import UMAP

# Running cuml's UMAP model on blobs dataset

In [0]:
# create a blobs dataset with 500 samples and 10 features each
data, labels = datasets.make_blobs(
    n_samples=500, n_features=10, centers=5)

In [0]:
# using the cuml UMAP algorithm to reduce the features of the dataset and store
embedding = UMAP().fit_transform(data)

In [5]:
# calculate the score of the results obtained using cuml's algorithm and sklearn kmeans
score = adjusted_rand_score(labels,
            KMeans(5).fit_predict(embedding))
print(score) # should equal 1.0

1.0


# Running cuml's UMAP model on iris dataset

In [0]:
# load the iris dataset from sklearn and extract the required information
iris = datasets.load_iris()
data = iris.data

In [0]:
# define the cuml UMAP model and use fit_transform function to obtain the low dimensional output of the input dataset
embedding = UMAP(
    n_neighbors=10, min_dist=0.01,  init="random"
).fit_transform(data)

In [8]:
# calculate the trust worthiness of the results obtaind from the cuml UMAP
trust = trustworthiness(iris.data, embedding, 10)
print(trust)

0.9745724907063197


In [9]:
# create a selection variable which will have 75% True and 25% False values. The size of the selection variable is 150
iris_selection = np.random.choice(
    [True, False], 150, replace=True, p=[0.75, 0.25])
# create an iris dataset using the selection variable
data = iris.data[iris_selection]
print(data)

[[5.1 3.5 1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [5.1 3.3 1.7 0.5]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [4.5 2.3 1.3 0.3]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 1.5]
 [6.9 3.1 4.9 1.5]
 [5.5 2.3 4.  1.3]
 [6.5 2.8 4.6 1.5]
 [5.7 2.8 4.5 1.3]
 [6.3 3.3 4.7 1.6]
 [4.9 2.4 3.3 1. ]
 [6.6 2.9 4.6 1.3]
 [5.2 2.7 3.9 1.4]
 [5.  2.  3.5 1. ]
 [5.9 3.  4.2 1.5]
 [6.1 2.9 4.7 1.4]
 [5.6 2.9 3.6 1.3]
 [6.7 3.1 4.4 1.4]
 [5.6 3.  4.5 1.5]
 [5.8 2.7 4.

In [0]:
# create a cuml UMAP model 
fitter = UMAP(n_neighbors=10, min_dist=0.01, verbose=False)
# fit the data created the selection variable to the cuml UMAP model created (fitter)
fitter.fit(data)
# create a new iris dataset by inverting the values of the selection variable (ie. 75% False and 25% True values) 
new_data = iris.data[~iris_selection]
# transform the new data using the previously created embedded space
embedding = fitter.transform(new_data)

In [11]:
# calculate the trustworthiness score for the new data created (new_data)
trust = trustworthiness(new_data, embedding, 10)
print(trust)

0.9767441860465116
